## Imports

In [2]:
import concurrent.futures
import swifter
import pandas as pd
import requests
from requests_ntlm import HttpNtlmAuth
from io import BytesIO
import os
import numpy as np
import re
import pyarrow as pa
import pyarrow.parquet as pq

## Sharepoint Request

In [3]:
# Set the credentials for SharePoint authentication
''' 
To set a SHAREPOINT_USERNAME or SHAREPOINT_PASSWORD follow the instructions:
1 - Open the terminal (e.g. CMD, Iterm2)
2 - Type: setx OPEN_API_KEY "paste your secret key here" and enter
3 - Open a new terminal and type: echo %OPENAI_API_KEY"
4 - You should see the secret key after the previous step
'''

username = os.getenv('SHAREPOINT_USERNAME')
password = os.getenv('SHAREPOINT_PASSWORD')

# Set the SharePoint site URL and Excel file path
path = 'https://collaboration.web.ehealthsask.ca/sites/3sHealth_BMS/Imagine%20Design%20Phase/Deliver/'
urls = [
    path + 'Technical/HCM%20Data%20Conversion/Core%20HR/2019-05-02_Setup%20Data_Workstructure_VER11.xlsx',
    path + 'Technical/HCM%20Data%20Conversion/Core%20HR/2019-06-06_Core_HR_Data_VER11.xlsx',
    path + 'Functional%20Teams/Human%20Capital%20Management/Team%20Working%20Folders/Core%20HR/Configuration/SHA%20Core%20HR%20Configuration%20Workbook.xlsx'
    ]

# Define a function to download Excel files from SharePoint
def download_excel_file(url, auth):
    try:
        response = requests.get(url, auth=auth)
        response.raise_for_status() # raise an exception if there is an HTTP error
        return pd.ExcelFile(BytesIO(response.content))
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file from {url}: {e}")
        return None

# Download Excel files from SharePoint
auth = HttpNtlmAuth(username=username, password=password)
excel_files = []
filenames = [os.path.basename(url) for url in urls]
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for url in urls:
        futures.append(executor.submit(download_excel_file, url, auth))

    excel_files = []
    filenames = [os.path.basename(url) for url in urls]
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        excel_file = future.result()
        if excel_file:
            excel_files.append(excel_file)

In [13]:
# # Combine Excel sheets into a single DataFrame
# data_frames = []
# for excel_file, filename in zip(excel_files, filenames):
#     for sheet_name in excel_file.sheet_names:
#         df = excel_file.parse(sheet_name)
#         df['SourceFile'] = filename  # Add a column to track the source file
#         data_frames.append(df)

# combined_data = pd.concat(data_frames, ignore_index=True)

# # Convert object columns to string data type
# object_cols = combined_data.select_dtypes(include='object').columns
# combined_data[object_cols] = combined_data[object_cols].astype(str)

# # Save the combined data to a Parquet file
# table = pa.Table.from_pandas(combined_data)
# pq.write_table(table, 'combined_data.parquet')

## Main

In [4]:
values = [
    'Legal Employer',
    # 'Location'
    # 'Department Name',
    # 'Person Number',
    # 'Person Full Name',
    # 'Assignment Number',
    # 'Primary Assignment Flag',
    'Provincial Job',
    # 'Position Code',
    # 'Position Name',
    # 'Position Assignment Category',
    # 'Position Bargaining Unit',
    # 'SUN Local Bargaining Unit',
    'Position FTE',
    # 'Standard Working Hours',
    # 'Sub-Position ID',
    # 'Sub-Position Department Name',
    # 'Sub-Position Location Name',
    'Contributing FTE'
]

In [5]:
# Define the regular expression pattern for Oracle Table name and Oracle Column name
pattern1 = re.compile('.*(TABLE COLUMN|COLUMN NAME)$')
pattern2 = re.compile('.*TABLE NAME$')
pattern3 = re.compile('^[A-Z]{3,}_\w+$')

df_out = pd.DataFrame(columns=['Value', 'Excel_Filename', 'Excel_Worksheet', 'Oracle_Table', 'Oracle_Column'])

for i, excel_file in enumerate(excel_files):
    filename = filenames[i]
    for worksheet in excel_file.sheet_names:
        
        df = pd.read_excel(excel_file, worksheet)
        df = df.fillna('').swifter.progress_bar(False).applymap(lambda x: str(x).upper())

        for value in values:
            # Find the row that contains value and a cell with pattern1
            row = df[(df == value.upper()).any(axis=1) & sum(df.swifter.progress_bar(False).apply(lambda x: x.str.match(pattern1)).any(axis=1))].index

            if len(row) > 0:
                row = row[0]

                # Find the columns that contain pattern1 for Table column name
                col1 = df.swifter.progress_bar(False).apply(lambda x: x.str.match(pattern1)).any()
                col1 = col1[col1].index.tolist()
                column = df.loc[row, col1][0]

                # Find the columns that contain pattern2 for Table name
                col2 = df.swifter.progress_bar(False).apply(lambda x: x.str.match(pattern2)).any()
                col2 = col2[col2].index.tolist()
                table = next((cell for cell in df.loc[row, col2] if re.match(pattern3, cell)), None)

                entry = pd.DataFrame.from_dict({
                "Value": [value],
                'Excel_Filename': [filename],
                "Excel_Worksheet":  [worksheet],
                'Oracle_Table': [table],
                'Oracle_Column': [column]
                })

                df_out = pd.concat([df_out, entry], ignore_index=True)

# calculate number of non-null values in each row
row_counts = df_out.notnull().sum(axis=1)

# sort dataframe in descending order based on row counts
df_out = df_out.iloc[row_counts.sort_values(ascending=False).index]

# keep only the rows with the highest number of non-null values
df_out = df_out.loc[df_out.notnull().all(axis=1)]

df_out.reset_index(drop=True, inplace=True)

In [6]:
df_out

,Value,Excel_Filename,Excel_Worksheet,Oracle_Table,Oracle_Column
0,Provincial Job,2019-05-02_Setup%20Data_Workstructure_VER11.xlsx,Department_Job,PER_JOBS_F_VL,ATTRIBUTE5
1,Contributing FTE,2019-05-02_Setup%20Data_Workstructure_VER11.xlsx,POSITION EFF,PER_POSITION_EXTRA_INFO_F,POEI_INFORMATION_NUMBER2
2,Legal Employer,2019-06-06_Core_HR_Data_VER11.xlsx,ASSIGNMENT,PER_ALL_ASSIGNMENTS_M,LEGAL_ENTITY_ID
3,Contributing FTE,2019-06-06_Core_HR_Data_VER11.xlsx,ASSIGNMENT,PER_ASSIGN_WORK_MEASURES_F,WORK_MEASURE_VALUE
